# Skorchを使って学習

- skorchを使うとsklearnと同じインターフェースが使え、学習ループ部分を自作する必要がない
- Gridsearchなど、sklearnの便利な機能も同様に用意されている

In [36]:
from pathlib import Path

import torch
from torch import optim
from torchvision import transforms, datasets
import torchvision.models as models

In [14]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

path = "dogscats/"
image_datasets = {x: datasets.ImageFolder(Path(path).joinpath(x),
                                          data_transforms[x])
                  for x in ['train', 'valid']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes
print(dataset_sizes, class_names)

{'train': 23000, 'valid': 2000} ['cats', 'dogs']


In [15]:
vgg16 = models.vgg16(pretrained=True)
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [16]:
# 置き換える層以外は再学習しないようにする
for param in vgg16.parameters():
    param.requires_grad = False

In [19]:
num_features = vgg16.classifier[6].in_features
modules = list(vgg16.classifier.children())
modules.pop()
modules.append(torch.nn.Linear(num_features, len(class_names)))
modules.append(torch.nn.Softmax(dim=1)) # 出力が確率値になっていることをNeuralNetClassifierは想定しているのでSoftmaxが必須
new_classifier = torch.nn.Sequential(*modules)
vgg16.classifier = new_classifier

In [20]:
image_datasets['train'][0][1]                        

0

## 学習部分

In [22]:
from skorch.callbacks import LRScheduler, Checkpoint

In [23]:
lrscheduler = LRScheduler(policy='StepLR', step_size=7, gamma=0.1)
checkpoint = Checkpoint(f_params='best_model.pt', monitor='valid_loss_best') # またはvalid_acc_bestを選ぶ
callbacks = [lrscheduler, checkpoint]

In [30]:
from skorch.helper import filtered_optimizer
from skorch.helper import filter_requires_grad
from skorch.helper import predefined_split

In [31]:
optimizer = filtered_optimizer(optim.SGD, filter_requires_grad) # finetuneなのでfilter_requires_gradが必要

[NeuralNet](https://skorch.readthedocs.io/en/latest/user/neuralnet.html)

In [ ]:
from skorch import NeuralNetClassifier

In [34]:
net = NeuralNetClassifier(
    vgg16,
    lr=0.001,
    batch_size=32,
    max_epochs=1,
    optimizer=optimizer,
    optimizer__momentum=0.9,
    iterator_train__shuffle=True,
    iterator_train__num_workers=4,
    iterator_valid__shuffle=False,
    iterator_valid__num_workers=4,
    train_split=predefined_split(image_datasets['valid']), # validを別途用意しているので渡す必要あり
    callbacks=callbacks,
    device='cuda'
)

In [35]:
net.fit(image_datasets['train'], y=None) # ImageFolderだとdatasetにxとy両方含まれるためy=Noneに

  epoch    train_loss    valid_acc    valid_loss    cp       dur
-------  ------------  -----------  ------------  ----  --------
      1        0.1201       0.9890        0.0306     +  348.3232


<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inpl

## 推論部分

In [37]:
new_net = NeuralNetClassifier(
    vgg16,
    lr=0.001,
    batch_size=32,
    max_epochs=1,
    device='cuda'
)
# iterator_valid__shuffleは絶対Falseにすること

In [38]:
new_net.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inpl

In [39]:
new_net.load_params('best_model.pt')

In [41]:
data_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
path = "dogscats/sample"
dataset = datasets.ImageFolder(root=os.path.join(path, 'valid'), transform=data_transform) # test1はラベル付されていないのでvalidで代替

In [42]:
y_proba = new_net.predict_proba(dataset)
y_proba

array([[9.9998105e-01, 1.8906063e-05],
       [9.9983454e-01, 1.6545040e-04],
       [9.9997354e-01, 2.6463878e-05],
       [1.6445540e-01, 8.3554459e-01],
       [4.3872471e-05, 9.9995613e-01],
       [1.9445532e-04, 9.9980551e-01],
       [2.7508903e-03, 9.9724907e-01],
       [6.3031012e-01, 3.6968994e-01]], dtype=float32)

In [43]:
predicted = new_net.predict(dataset)
predicted

array([0, 0, 0, 1, 1, 1, 1, 0])

In [44]:
y_test = [record[1] for record in dataset]
y_test

[0, 0, 0, 0, 1, 1, 1, 1]

In [46]:
import numpy as np
np.set_printoptions(precision=4, linewidth=100)

In [47]:
np.mean(predicted == y_test)

0.75

## GridSearch

Gridsearchではxとyを別々に持っていないと厳しいので、ImageFolderを使う場合は加工が必要

In [ ]:
x = [record[0] for record in image_datasets['train']]
y = [record[1] for record in image_datasets['train']]

In [ ]:
x[0].size()

In [ ]:
X = torch.stack(x)
X.size()

In [ ]:
y = torch.tensor(y)

In [ ]:
from skorch.dataset import get_len

In [ ]:
print(get_len(X), get_len(y))

In [ ]:
net = NeuralNetClassifier(
    vgg16,
    lr=0.001,
    batch_size=32,
    max_epochs=3,
    optimizer=optimizer,
    optimizer__momentum=0.9,
    iterator_train__shuffle=True,
    iterator_train__num_workers=4,
    iterator_valid__shuffle=False,
    iterator_valid__num_workers=4,
    train_split=predefined_split(image_datasets['valid']),
    callbacks=callbacks,
    device='cuda'
)

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    'lr': [0.001],
    'max_epochs': [2]
}
gs = GridSearchCV(net, params, cv=2, refit=False, scoring='accuracy')
gs.fit(X.numpy(), y.numpy())
print(gs.best_score_, gs.best_params_)